# CLI Generator

Fetches a documentation URL and uses an LLM to generate a Python CLI tool.

**Interactive use** – edit the *Parameters* cell below and run all cells.  
**Headless use** – `make cli URL=https://... [PROVIDER=claude|hf] [MODEL=...]`

Output is saved to `cli/<domain>/<model>/cli.py`.

In [ ]:
# ── Parameters ────────────────────────────────────────────────────────────────
# Edit here for interactive use; overridden automatically by `make cli`.

# Provider: "claude" or "hf"
PROVIDER = "claude"

# Claude models : claude-opus-4-6 | claude-sonnet-4-6 | claude-haiku-4-5-20251001
# HF models     : mistralai/Mistral-7B-Instruct-v0.3 | meta-llama/Meta-Llama-3-8B-Instruct
MODEL = "claude-opus-4-6"

# Documentation URL to generate a CLI from
URL = ""

In [ ]:
import os
import re
from pathlib import Path
from urllib.parse import urlparse

from dotenv import load_dotenv

load_dotenv(dotenv_path=Path("../.env"))  # works from notebooks/ or repo root
load_dotenv()                              # fallback: CWD .env

# Fall back to environment variables when parameters were not set by papermill
PROVIDER = PROVIDER or os.getenv("PROVIDER", "claude")
MODEL    = MODEL    or os.getenv("MODEL",    "claude-opus-4-6")
URL      = URL      or os.getenv("URL",      "")

assert URL, (
    "URL must be set.\n"
    "  Interactive : edit the Parameters cell above.\n"
    "  Headless    : make cli URL=https://..."
)

print(f"Provider : {PROVIDER}")
print(f"Model    : {MODEL}")
print(f"URL      : {URL}")

In [ ]:
import requests
from bs4 import BeautifulSoup

resp = requests.get(URL, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")
for tag in soup(["script", "style", "nav", "footer", "header", "aside"]):
    tag.decompose()

page_text = soup.get_text(separator="\n", strip=True)
page_text = "\n".join(line for line in page_text.splitlines() if line.strip())
page_text = page_text[:12_000]  # keep within model token budget

print(f"Fetched {len(page_text):,} chars from {URL}")

In [ ]:
SYSTEM = "You are an expert Python developer. Write clean, idiomatic, production-quality Python code."

USER = f"""Based on the documentation below, create a complete, working Python CLI tool using argparse.

Requirements:
- Implement the main functionality described in the docs
- Use argparse with clear --help text for every argument
- Include a main() function and `if __name__ == '__main__': main()`
- Add concise error handling
- Every function must be fully implemented — no placeholders

Source: {URL}

Documentation:
---
{page_text}
---

Respond with ONLY the Python code, starting with the imports."""

print(f"Prompt built ({len(USER):,} chars). Calling {PROVIDER}/{MODEL}...")

In [ ]:
if PROVIDER == "claude":
    import anthropic

    client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
    response = client.messages.create(
        model=MODEL,
        max_tokens=4096,
        system=SYSTEM,
        messages=[{"role": "user", "content": USER}],
    )
    raw = response.content[0].text

elif PROVIDER == "hf":
    from huggingface_hub import InferenceClient

    client = InferenceClient(token=os.getenv("HF_TOKEN"))
    result = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user",   "content": USER},
        ],
        max_tokens=2048,
        temperature=0.1,
    )
    raw = result.choices[0].message.content

else:
    raise ValueError(f"Unknown PROVIDER '{PROVIDER}'. Use 'claude' or 'hf'.")

print(raw[:800] + ("\n..." if len(raw) > 800 else ""))

In [ ]:
# Strip markdown code fences if the model wrapped the code
code_match = re.search(r"```(?:python)?\n(.*?)```", raw, re.DOTALL)
cli_code = code_match.group(1).strip() if code_match else raw.strip()

# Output path: cli/<domain>/<model-slug>/cli.py
domain     = urlparse(URL).netloc.lstrip("www.")
model_slug = MODEL.replace("/", "_").replace(":", "_")

# Resolve relative to repo root regardless of CWD
repo_root = Path("__file__").parent.parent if Path("__file__").exists() else Path(".")
# Use CWD-relative path (works from repo root for both jupyter and papermill)
out_dir = Path("cli") / domain / model_slug
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "cli.py"
out_path.write_text(cli_code, encoding="utf-8")

print(f"\nCLI tool saved  → {out_path.resolve()}")
print(f"Run with        : python {out_path} --help")